In [64]:
from feature_detection.darkflow.net.build import TFNet
import cv2
import pandas as pd
import labelscount
from IPython.display import Image
import matplotlib.pyplot as plt
import math


# Load TFNet

In [ ]:
options = {"model": "./feature_detection/cfg/road_yolo.cfg",
           "config": "./feature_detection/cfg",
           "load": -1,
           "batch" : 4,
           "labels": "./feature_detection/labels.txt",
           'backup': "./feature_detection/model",
           "gpu": 0.0}

light_windows_tfnet = TFNet(options)

In [ ]:
car_options = {"model": "./feature_detection/cfg/yolo.cfg", 
           "load": "./feature_detection/bin/yolo.weights",
           "config": "./feature_detection/cfg",
           "batch" : 4,
           "labels": "./feature_detection/labels.txt",
           "gpu": 0.0}

# options
car_tfnet = TFNet(car_options)

# Read Crime and Safe file

In [ ]:
# Paths
crime_path = './dataset/crime_locations.csv'
safe_path = './dataset/safe_locations.csv'

In [ ]:
crime_df = pd.read_csv(crime_path)
crime_df

In [55]:
safe_df = pd.read_csv(safe_path)
safe_df

,id,windows,car,lights,gas,convience,sdpd,lampdist,lat,lon,crime
0,s_0,0,0,2,0,0,0,15.491717,32.594749,-117.084097,safe
1,s_1,0,0,0,0,0,0,43.872941,32.584117,-117.073782,safe
2,s_2,0,0,0,0,0,0,70.877100,32.584009,-117.065199,safe
3,s_3,0,0,0,0,0,0,25.571291,32.557387,-117.084123,safe
4,s_4,0,0,0,0,0,0,39.372835,32.546825,-117.065841,safe
5,s_5,0,0,1,0,0,0,18.176018,32.565706,-117.049876,safe
6,s_6,0,0,0,0,0,0,53.970956,32.561149,-117.049361,safe
7,s_7,0,0,0,0,0,0,33.506199,32.571493,-117.058202,safe
8,s_8,0,0,0,0,0,0,37.882438,32.583861,-117.052537,safe
9,s_9,0,0,0,0,0,0,48.860448,32.577778,-117.045437,safe


In [49]:
# crime_df = crime_df[crime_df.id != "2011_93602"]
# # crime_df.at[588,"id"]


# Start to detect the crime street view.

In [69]:
# Scan pictures
streetview_path = './dataset/crime_streetview/'
pic_num = 4;

for index in range(len(crime_df)):
    crime_id = crime_df.at[index,"id"]
    car_sum = 0
    for num in range(pic_num):
        picture_name = crime_id+"_" + str(num) + ".jpg"
        street_img = cv2.imread(streetview_path+picture_name)
        street_img = cv2.cvtColor(street_img, cv2.COLOR_BGR2RGB)

        scan_results = light_windows_tfnet.return_predict(street_img)
        window_result = [aresult for aresult in scan_results if aresult['label'] == "window" and aresult['confidence'] > 0.1]
        final_result = labelscount.mergeoverlaylabel(window_result)
#         crime_df.at[index,"windows"]  =len(final_result)
        crime_df.at[index,"windows"]  +=len(final_result)

        
        scan_results = car_tfnet.return_predict(street_img)
        car_result = [aresult for aresult in scan_results if aresult['label'] == "car" and aresult['confidence'] > 0.1]
        final_result = labelscount.mergeoverlaylabel(car_result)
#         crime_df.at[index,"car"]  =len(final_result)
        car_sum += len(final_result)
    
    crime_df.at[index,"car"]  =math.ceil(car_sum/pic_num)

# crime_df


In [70]:
crime_df.to_csv("./dataset/crime_locations.csv", index=False)
# crime_df

# Start to detect the safe street view.

In [71]:
# Scan pictures
streetview_path = './dataset/safe_streetview/'
pic_num = 4;

for index in range(len(safe_df)):
    safe_id = safe_df.at[index,"id"]
    car_sum = 0
    for num in range(pic_num):
        picture_name = safe_id+"_" + str(num) + ".jpg"
        street_img = cv2.imread(streetview_path+picture_name)
        street_img = cv2.cvtColor(street_img, cv2.COLOR_BGR2RGB)

        scan_results = light_windows_tfnet.return_predict(street_img)
        window_result = [aresult for aresult in scan_results if aresult['label'] == "window" and aresult['confidence'] > 0.1]
        final_result = labelscount.mergeoverlaylabel(window_result)
        safe_df.at[index,"windows"]  +=len(final_result)
        
        scan_results = car_tfnet.return_predict(street_img)
        car_result = [aresult for aresult in scan_results if aresult['label'] == "car" and aresult['confidence'] > 0.1]
        final_result = labelscount.mergeoverlaylabel(car_result)
        car_sum += len(final_result)
    
    safe_df.at[index,"car"]  =math.ceil(car_sum/4)

safe_df

,id,windows,car,lights,gas,convience,sdpd,lampdist,lat,lon,crime
0,s_0,0,7,2,0,0,0,15.491717,32.594749,-117.084097,safe
1,s_1,0,1,0,0,0,0,43.872941,32.584117,-117.073782,safe
2,s_2,0,4,0,0,0,0,70.877100,32.584009,-117.065199,safe
3,s_3,0,1,0,0,0,0,25.571291,32.557387,-117.084123,safe
4,s_4,0,1,0,0,0,0,39.372835,32.546825,-117.065841,safe
5,s_5,0,1,1,0,0,0,18.176018,32.565706,-117.049876,safe
6,s_6,0,0,0,0,0,0,53.970956,32.561149,-117.049361,safe
7,s_7,0,2,0,0,0,0,33.506199,32.571493,-117.058202,safe
8,s_8,0,6,0,0,0,0,37.882438,32.583861,-117.052537,safe
9,s_9,0,4,0,0,0,0,48.860448,32.577778,-117.045437,safe


In [72]:
safe_df.to_csv("./dataset/safe_locations.csv", index=False)
